In [28]:
from langchain.chat_models import init_chat_model
## create llm
llm = init_chat_model("gpt-5-nano")

In [29]:
llm.invoke("Hello")

AIMessage(content='Hi there! How can I help today?\n\nHere are a few things I can do:\n- Explain concepts or answer questions\n- Draft, edit, or summarize text\n- Write or debug code\n- Translate or summarize documents\n- Brainstorm ideas or plan a project\n- Plan a trip, meal plan, or workout\n- Help with math, science, history, and more\n\nTell me your goal, any constraints, and your preferred language (if any), and I’ll get started. What would you like to do?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 564, 'prompt_tokens': 7, 'total_tokens': 571, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 448, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CxPtC3DoGsNB3iBQcosni2JRrAulW', 'service_tier': 'default', 'finish_reason

In [30]:
# add tools
from langchain.tools import tool

@tool
def get_revenue():
    """Retreive the revenue till now"""
    return f"The revenue till now is $1000"

@tool
def get_active_users():
    """Return all current active users"""
    return "There are 5 active users"

In [31]:
llm_with_tool = llm.bind_tools([get_revenue, get_active_users], tool_choice="any", parallel_tool_calls=False)

In [32]:
llm_with_tool.invoke("get me  all active users?").tool_calls

[{'name': 'get_active_users',
  'args': {},
  'id': 'call_voR44gTufDtHB9pkxmynJTUr',
  'type': 'tool_call'}]

### Steps
 - Email wil come
 - (EMAIL ROUTER) system will categories (decide)
     - respond
     - notify
     - ignore
-  if respond (AGENT)
    - forward that mail to agent


In [33]:
email_input = {
  "author": "Alice Smith <alice.smith@company.com>",
  "to": "John Doe <john.doe@company.com>",
  "subject": "Quick question about API documentation",
  "email_thread": "Hi John,\nI was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\nSpecifically, I'm looking at:\n- /auth/refresh\n- /auth/validate\nThanks!\nAlice"
}


In [34]:
from email_assistant.utils import parse_email
from email_assistant.prompts import triage_user_prompt
from rich.markdown import Markdown

author, to, subject, email_thread = parse_email(email_input)

In [40]:
user_prompt = triage_user_prompt.format(author=author, to=to,subject=subject, email_thread=email_thread)

In [36]:
from langgraph.graph import MessagesState
from typing import Literal

class State(MessagesState):
    email_input:dict
    classification_decision :Literal["ignore", "notify", "respond"]

In [37]:
from pydantic import BaseModel, Field
class RouterSchema(BaseModel):
    """Analyse the email and classtifybased on its content"""
    reasoning:str = Field(description="Step by Step reason behind the classification")
    classification: Literal["ignore", "notify", "respond"] = Field(description="The classification of an email: 'ignore' for irrelevant emails, "
        "'notify' for important information that doesn't need a response, "
        "'respond' for emails that need a reply",)


In [38]:
llm_with_struct = llm.with_structured_output(RouterSchema)

In [43]:
llm_with_struct.invoke(user_prompt)

RouterSchema(reasoning='Alice asks John to clarify whether the endpoints /auth/refresh and /auth/validate are intentional omissions or missing from the docs, which is an actionable item requiring a direct reply from John or the API/docs owner. This is not spam or general info, so it should be handled with a response and a plan to verify and possibly update the docs.', classification='respond')

In [ ]:
from langgraph.types import Command

def traige_router(state:State):
    author, to, subject, email_thread = parse_email(state["email_input"])
    user_prompt = triage_user_prompt.format(author=author, to=to,subject=subject, email_thread=email_thread)    
    out = llm_with_struct.invoke(user_prompt)

    if out.classification == "respond": 
        goto ="response_agent"
        update = {
            "messages" : [{
                "role" : "user",
                "content" : f"Respond to the email: \n\n{format_email_markdown(subject, author, to, email_thread)}"
            }],
            "classification_decision" : "respond"
        }
    elif out.classification == "notify":
        goto= END
        update = {
            "classification_decision" : "ignore"
        }
    elif out.classification == "ignore":
        goto= END
        update = {
            "classification_decision" : "ignore"
        }
    else: 
        raise ValueError("value Error")
    
    return Command(goto=goto, update=update)



In [48]:
from typing import Literal
from datetime import datetime
from pydantic import BaseModel
from langchain_core.tools import tool

@tool
def write_email(to: str, subject: str, content: str) -> str:
    """Write and send an email."""
    # Placeholder response - in real app would send email
    return f"Email sent to {to} with subject '{subject}' and content: {content}"

@tool
def schedule_meeting(
    attendees: list[str], subject: str, duration_minutes: int, preferred_day: datetime, start_time: int
) -> str:
    """Schedule a calendar meeting."""
    # Placeholder response - in real app would check calendar and schedule
    date_str = preferred_day.strftime("%A, %B %d, %Y")
    return f"Meeting '{subject}' scheduled on {date_str} at {start_time} for {duration_minutes} minutes with {len(attendees)} attendees"

@tool
def check_calendar_availability(day: str) -> str:
    """Check calendar availability for a given day."""
    # Placeholder response - in real app would check actual calendar
    return f"Available times on {day}: 9:00 AM, 2:00 PM, 4:00 PM"

@tool
class Done(BaseModel):
      """E-mail has been sent."""
      done: bool

In [49]:
tools = [write_email, schedule_meeting, check_calendar_availability, Done]


In [53]:
tools_by_name = {tool.name: tool for tool in tools}
